# Test du modèle

Pré-requis :
- avoir exécuté le fichier preprocessing

**L'entrainement du modèle n'est pas nécessaire**

On va ici tester et visualiser dans un dossier "predicted_images" les résultats de notre modèle

In [1]:
#### BIBLIOTHEQUES ####

import os
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers, models

2024-11-17 23:01:04.721926: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-17 23:01:04.732892: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-17 23:01:04.832879: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-17 23:01:04.935110: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-17 23:01:05.042201: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been 

In [2]:
#### MODELE U-NET ####

def unet_model_with_water_body(input_shape=(256, 256, 1)):
    """
    Build a U-Net model that takes in VV, VH polarization images and 
    the water body label as input to predict the flooded areas.
    
    Input: (VV, VH, Water Body Label) -> 3 channels
    Output: Flooded area segmentation -> 1 channel
    """

    inputs = layers.Input(shape=input_shape)

    # Encoder
    c1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    c1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c1)
    p1 = layers.MaxPooling2D((2, 2))(c1)

    c2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(p1)
    c2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c2)
    p2 = layers.MaxPooling2D((2, 2))(c2)

    # Decoder
    u1 = layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(p2)
    u1 = layers.concatenate([u1, c2])
    c3 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(u1)
    c3 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c3)

    u2 = layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c3)
    u2 = layers.concatenate([u2, c1])
    c4 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(u2)
    c4 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c4)

    # Output layer (flooded area prediction)
    outputs = layers.Conv2D(1, (1, 1), activation='sigmoid')(c4)

    # Compile the model
    model = models.Model(inputs=[inputs], outputs=[outputs])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    return model


model = unet_model_with_water_body()
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 256, 256,  │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 256, 256,  │        640 │ input_layer[0][0] │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 256, 256,  │     36,928 │ conv2d[0][0]      │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 128, 128,  │          0 │ conv2d_1[0][0]    │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 128, 128,  │     73,856 │ max_pooling2d[0]… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 128, 128,  │    147,584 │ conv2d_2[0][0]    │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 64, 64,    │          0 │ conv2d_3[0][0]    │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_transpose    │ (None, 128, 128,  │     65,664 │ max_pooling2d_1[… │
│ (Conv2DTranspose)   │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 128, 128,  │          0 │ conv2d_transpose… │
│ (Concatenate)       │ 256)              │            │ conv2d_3[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 128, 128,  │    295,040 │ concatenate[0][0] │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_5 (Conv2D)   │ (None, 128, 128,  │    147,584 │ conv2d_4[0][0]    │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_transpose_1  │ (None, 256, 256,  │     32,832 │ conv2d_5[0][0]    │
│ (Conv2DTranspose)   │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_1       │ (None, 256, 256,  │          0 │ conv2d_transpose… │
│ (Concatenate)       │ 128)              │            │ conv2d_1[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_6 (Conv2D)   │ (None, 256, 256,  │     73,792 │ concatenate_1[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_7 (Conv2D)   │ (None, 256, 256,  │     36,928 │ conv2d_6[0][0]    │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_8 (Conv2D)   │ (None, 256, 256,  │         65 │ conv2d_7[0][0]    │
│                     │ 1)                │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 910,913 (3.47 MB)

 Trainable params: 910,913 (3.47 MB)

 Non-trainable params: 0 (0.00 B)

## Ne rien modifier ci-dessus

On vient de définir le modèle et de charger les paramètres d'entrainement associés à une epoch. Il reste à charger des données pour faire des prédictions. 

Pour le moment le code ci-dessous ne marche pas car il faut charger les données dans le bash Python au préalable. L'idéal serait de ne pas avoir à charger toute la base de données mais juste les données sur lesquelles on souhaite faire une prédiction

In [3]:
#### CHOISIR L'EPOCH A TESTER ####
epoch = '10' # de 01 à 20

#### CHOISiR LE MODELE A TESTER #### (1 ou 3, correspondant au nombre de dossiers d'images de la BDD d'entrainement)
dossier = '3'
model.load_weights('./model_parameters_' + dossier +'/model_epoch_'+epoch+'.weights.h5')

/home/maelh/ML_projects/NASA-Flood-Extent-Detection/.venv/lib/python3.12/site-packages/keras/src/saving/saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 46 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [4]:
import cv2
import glob

# Define directories
vv_dir = '../Final_database/vv/'
vh_dir = '../Final_database/vh/'
water_body_dir = '../Final_database/water_body_label/'
flood_label_dir = '../Final_database/flood_label/'
preprocessed_dir = '../preprocessed_data/'


def load_images_from_folder(folder, label_folder):
    # Initialiser les tableaux NumPy avec la taille requise
    num_images = len(glob.glob(os.path.join(folder, '*.png')))
    images = np.zeros((num_images, 256, 256), dtype=np.uint8)
    labels = np.zeros((num_images, 256, 256), dtype=np.uint8)

    for i, filename in enumerate(glob.glob(os.path.join(folder, '*.png'))):
        # Charger l'image en uint8
        img = cv2.imread(filename, cv2.IMREAD_GRAYSCALE)
        if img is None:
            print(f"Erreur de chargement de l'image: {filename}")
            continue  # Passer à l'image suivante
        
        img = cv2.resize(img, (256, 256))  # Redimensionner
        images[i] = img  # Remplir le tableau NumPy
        
        # Charger le label correspondant
        label_name = os.path.basename(filename).replace('_vv.png', '.png')
        label_path = os.path.join(label_folder, label_name)
        label_img = cv2.imread(label_path, cv2.IMREAD_GRAYSCALE)
        if label_img is None:
            print(f"Erreur de chargement du label: {label_path}")
            continue  # Passer au label suivant
        
        label_img = cv2.resize(label_img, (256, 256))
        binary_label = (label_img > 0).astype(np.uint8)
        labels[i] = binary_label  # Remplir le tableau NumPy

    # Retirer les images et labels qui n'ont pas pu être chargés
    valid_indices = np.where(images.sum(axis=(1, 2)) > 0)[0]
    return images[valid_indices], labels[valid_indices]

# Exemple d'utilisation



images, labels = load_images_from_folder(vv_dir,water_body_dir)

X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.2, random_state=42)

In [14]:
##### PREDICTION DU MODELE #####
# After training, make predictions on a subset of the validation set (3 samples)
y_pred = model.predict(X_val[:10])
y_mask = np.where(y_pred > 0.03, 1, 0)
print(y_mask.shape)
y_temp = y_mask.reshape((10, 256, 256))

# Calculate the IoU metric for each prediction
iou_scores = []
for k in range(10):
    sum=0
    iou_score_image = 0
    for i in range(256):
        for j in range(256):
            if y_temp[k][i][j] == 1 :
                sum +=1

                if y_val[k][i][j] == y_temp[k][i][j]:
                    iou_score_image+=1

    if sum !=0 :
        iou_scores.append(iou_score_image/sum)

    print(iou_scores)

print("Average IoU score for 3 samples:", np.mean(iou_scores))

/home/maelh/ML_projects/NASA-Flood-Extent-Detection/.venv/lib/python3.12/site-packages/keras/src/models/functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['keras_tensor']. Received: the structure of inputs=*
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
(10, 256, 256, 1)
[0.0]
[0.0, 0.3053206712631716]
[0.0, 0.3053206712631716]
[0.0, 0.3053206712631716, 0.14583333333333334]
[0.0, 0.3053206712631716, 0.14583333333333334, 0.07537688442211055]
[0.0, 0.3053206712631716, 0.14583333333333334, 0.07537688442211055, 0.6542962219993314]
[0.0, 0.3053206712631716, 0.14583333333333334, 0.07537688442211055, 0.6542962219993314, 0.6254826254826255]
[0.0, 0.3053206712631716, 0.14583333333333334, 0.07537688442211055, 0.6542962219993314, 0.6254826254826255, 0.0]
[0.0, 0.3053206712631716, 0.14583333333333334, 0.07537688442211055, 0.6542962219993314, 0.6254826254826255, 0.0]
[0.0, 0.3053206712631716, 0.14583333333333334, 0.07537688442211055, 0.6542962219993314, 0.6254826254826255, 0.0, 0.06918238993710692]
Average IoU score for 3 samples: 0.23443651580470992


In [34]:
##### IMPRIME LES IMAGES EN PNG ####
import imageio

# Ensure the output directory exists
output_dir = '../predicted_images'
os.makedirs(output_dir, exist_ok=True)

# Transform the 3 predicted images into PNG files
for i in range(10):
    # Rescale the image to 0-255 and convert to uint8
    img = (y_mask[i, :, :, 0] * 255).astype(np.uint8)
    # Save the image as a PNG file
    imageio.imwrite(os.path.join(output_dir, f'predicted_image_{i}.png'), img)